# GAN을 만들어봅시다 갠갠갠 갠 널 사랑하지 않아

### class 만드는데 궁금리스트
#### Batch Normalization Momentum
#### initial Dense Layer
#### upsample
#### z_dim

In [7]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dropout, Flatten, UpSampling2D

In [6]:
class GAN():
    def __init__(self
        , input_dim #input 값의 dimension 일단 여기 animal GAN에서는 28 * 28 * 흑백(1) 짜리 사용
        , discriminator_conv_filters # conv_filter 64 * 64 * 128 * 128 - filter가 총 네층인데 64,64,128,128로 하겠다는 뜻
        , discriminator_conv_kernel_size # kernel_size 5 * 5* 5* 5
        , discriminator_conv_strides # stride 2* 2* 2* 1
                 
        , discriminator_batch_norm_momentum # Batch Normalization 하냐 안하냐 인가 이게
        , discriminator_activation # Activation function
        , discriminator_dropout_rate # dropout_rate 층 깔면 얼마나 ?? 
        , discriminator_learning_rate # lr
                 
        , generator_initial_dense_layer_size # 왜 intital dense_layer_size지
        , generator_upsample # upsamle의 사이즈를 말하는 건가요 2*2*1*1 이긴하네요
        , generator_conv_filters #conv의 필터
        , generator_conv_kernel_size #kernel size
        , generator_conv_strides # stride
        , generator_batch_norm_momentum # batch_norm_momentum 모멘텀이네요 하냐 안하냐가 아니라 0.9로 되어있는데 왜지
        , generator_activation # Activation function
        , generator_dropout_rate 
        , generator_learning_rate
        , optimiser
        , z_dim # 다시 모르겠는 z_dim
        ):

        self.name = 'gan' # 이름은 Gan 이라고 지을께요

        self.input_dim = input_dim 
        self.discriminator_conv_filters = discriminator_conv_filters
        self.discriminator_conv_kernel_size = discriminator_conv_kernel_size
        self.discriminator_conv_strides = discriminator_conv_strides
        self.discriminator_batch_norm_momentum = discriminator_batch_norm_momentum
        self.discriminator_activation = discriminator_activation
        self.discriminator_dropout_rate = discriminator_dropout_rate
        self.discriminator_learning_rate = discriminator_learning_rate

        self.generator_initial_dense_layer_size = generator_initial_dense_layer_size
        self.generator_upsample = generator_upsample
        self.generator_conv_filters = generator_conv_filters
        self.generator_conv_kernel_size = generator_conv_kernel_size
        self.generator_conv_strides = generator_conv_strides
        self.generator_batch_norm_momentum = generator_batch_norm_momentum
        self.generator_activation = generator_activation
        self.generator_dropout_rate = generator_dropout_rate
        self.generator_learning_rate = generator_learning_rate
        
        self.optimiser = optimiser
        self.z_dim = z_dim

        
        self.n_layers_discriminator = len(discriminator_conv_filters) # discriminator의 layer 몇 개 깔지 위의 64,64,128,128 총 네 개
        self.n_layers_generator = len(generator_conv_filters) # generator 또한 같다

        self.weight_init = RandomNormal(mean=0., stddev=0.02) # 초기 가중치를 0 짜리 표준편차 0.02로 만들어봅시다

        self.d_losses = []  # 로스 리스트
        self.g_losses = [] # generator 로스 리스트

        self.epoch = 0

        self._build_discriminator() # 시작과 동시에 discriminator, generator, 만들고
        self._build_generator()

        self._build_adversarial() # adversarial --> 이거 D, G 연결한거겠지??
        
        # Activation Function 치면 그거 따라서 나와야하는 거 같은데 keras의 Activation Layer가 어케 작동하는거지?? ㅇㄹ
        # keras.layer.Activation은 앞선 층의 Activation argument를 입력해주는 듯하다. 근데 왜 굳이 Leaky_Relu만 저렇게 칠꺼라고 생각하는 걸까 뿌우 
        # 걍 알아서 하라고 하자
        
   # def get_activation(self, activation): 
    #    if activation == 'leaky_relu':
     #       layer = LeakyReLU(alpha = 0.2)
      #  else:
       #     layer = Activation(activation)
       # return layer
    
    
    def _build_discriminator(self):
        discriminator_input = Input(shape=self.input_dim, name= "discriminator_input") # Input 값 self 에서 받아서 쓰고
        x = discriminator_input
        
        
        for i in range(self.n_layers_discriminator): # discriminator 레이어층 수만큼
            
            x = Conv2D(
                filters = self.discriminator_conv_filters[i]
                , kernel_size = self.discriminator_conv_kernel_size[i]
                , strides = self.discriminator_conv_strides[i]
                , padding = 'same'
                , name = 'discriminator_conv_'+str(i)
            )(x)
            if self.discriminator_batch_norm_momentum and i > 0: # 0번째는 안만들어?? Conv2D 밑에,
                # 보통 Batch Normalization 은 fully connected 나 Conv2D 다음, Activation 전에 들어온다 
                
                x = BatchNormalization(momentum = self.discriminator_batch_norm_momentum)(x)
            
            x = Activation(self.discriminator_activation)(x)
            
            if self.discriminator_dropout_rate :
                x = Dropout(rate=self.discriminator_dropout_rate)(x)
            
        # 레이어세트 수만큼 Conv + Batch + Activation + Dropout 을 깔았어
        
        # 마지막으로 펴주고
        x = Flatten()(x)
        
        # Dense Layer를 sigmoid로 통과시켜서, output 출력 output shape 어케 되냐
        discriminator_output = Dense(1, activation ='sigmoid', kernel_initializer = self.weight_init)(x)
        
        discriminator = Model(discriminator_input, discriminator_output)
        
        
        
        
        
        
        
        
        def _build_generator(self):
            generator =Input(shape=(self.z_dim,), name = 'generator_input')
            x = generator_input
            
            x = Dense(np.prod(self.generator_initial_dense_layer_size))(x)
        # np.prod --> 각 항들의 값을 전부 곱한거 그니까 여기서 7*7*64로 generator initial dense layer를 잡았는데... 일단 잡았어
        
        
        if self.generator_batch_norm_momentum :
            x= BatchNormalization(momentum = self.generator_batch_norm_momentum)(x)
        x = Activation(self.generator_activation)(x)
        
        # 맨처음에 배치정규화랑 Relu를 통과시키고 그걸 7 7 64 텐서로 바꿔
        
        x = Reshape(self.generator_initial_dense_layer_size)(x)
        
        if self.generator_dropout_rate :
            x = Dropout(rate =self.generator_dropout_rate)(x)
            
        for i in range(self.n_layers_generator):
            
            x = UpSampling2D()(x)
            x = Conv2D(
            filters = self.generator_conv_filters[i],
            kernel_size= self.generator_conv_kernel_size[i],
            padding = 'same',
            name = 'generator_conv_'+str(i))(x)
            
            if i < n_layers_generator -1 :
                if self.generator_batch_norm_momentum:
                    x = BatchNormalization(momentum =self.generator_batch_norm_momentum)(x)
                x = Activation('relu')(x)
            else : Activation('tanh')(x)
            
    
    # 4개의 Conv2D 층 - 처음 두 개는 Upsampling 뒤에 놓입니다. 는 깃헙이고 책껄로 따라하니까 걍 Upsamling 4개 해보리네용
    # 섞어서 해도 되나 봅니당
    
        generator_output =x
        generator = Model(generator_input, generator_output)

        
        

        
        
    def get_opti(self, lr):
        if self.optimiser == 'adam':
            opti = Adam(lr=lr, beta_1=0.5)
        elif self.optimiser == 'rmsprop':
            opti = RMSprop(lr=lr)
        else:
            opti = Adam(lr=lr)

        return opti
    
    
    def _build_adversarial(self):
        
        ### COMPILE DISCRIMINATOR

        self.discriminator.compile(
        optimizer=self.get_opti(self.discriminator_learning_rate)  
        , loss = 'binary_crossentropy'
        ,  metrics = ['accuracy']
        )
        

        self.set_trainable(self.discriminator, False)

        model_input = Input(shape=(self.z_dim,), name='model_input')
        model_output = self.discriminator(self.generator(model_input))
        self.model = Model(model_input, model_output)

        self.model.compile(optimizer=self.get_opti(self.generator_learning_rate) , loss='binary_crossentropy', metrics=['accuracy'])

        self.set_trainable(self.discriminator, True)

        
        ### 늘 여기까지는 잘하지 모델 짜는 건 쉬운 편이야 하지만 나는 compile 이랑 학습을 못하지 후후
        
        
        
        
    

## BN Batch Normalization
#### 1. mean = 0 stev = 1 인 NP로 정규화
#### 2. scaling 과 shifting
그냥 저 NP로 만들어주면 sigmoid를 지날 때 값이 0인애가 너무 많기 때문에 sigmoid를 지나면 사실 상 해당 구간에서 선형의 모습을 보인다. 분명 비선형 데이터를 넣은건데도 --> 너무 0 근처에 있지 말라고 가중치 r (scaling) b (shifting)을 더해 값을 변형해준다.

#### 3. 장점
1. tanh나 sigmoid 같은 활성화 함수에 대해 그래디언트 소실(vanishing gradient)문제가 감소한다.

2. 가중치 초기화에 덜 민감하다. 가중치 초기값에 크게 의존하지 않기 때문에 05-1. 심층 신경망 학습에서 알아본 가중치 초기화 기법에 대해 크게 신경 쓰지 않아도 된다.

3. 학습률(learning rate)를 크게 잡아도 gradient descent가 잘 수렴한다.

4. 오버피팅을 억제한다. BN이 마치 Regularization 역할을 하기 때문에 드롭아웃(Dropout)과 같은 규제기법에 대한 필요성이 감소한다.  하지만, BN로 인한 규제는 효과가 크지 않기 때문에 드롭아웃을 함께 사용하는 것이 좋다.


#### 4. Momentum
우리가 학습을 할 때 Train 할 때는 Batch 들의 평균과 분산을 구할 수가 있는데, 새로운 데이터가 들어왔어. 얘를 어떻게 정규화를 시킬꺼야?? 평균과 분산이 있어야할 수 있느데 없잖아. 그래서 미리 학습 시에 이동평균(Moving Average)를 저장해둬
이 이동평균을 계산하기 위해 다시, 이동평균을 계산하기위해 --> 새로운 이동평균이 주는 영향을 제어하기 위해 momentum을 사용하고, 보통 momentum은 0.99 --> v = v * momentum + v_new * (1 - momentum)

## UpSampling Layer
VAE 에서는 Conv2DTranpose를 써서, Decode해줬는데, - 층마다 텐서의 너비와 높이를 두 배로 늘렸는데, [upsampling은 합성곱연산을 수행하기 전에 중간에 0을 채워] 왜죠 ? 뭐 둘 다 좋은데 뭐쓰는지는 니가 알아서해 Conv2DTranspose랑 Upsamling 둘 다 써